# COVID19 evolution analysis

#### data loading

In [229]:
%load_ext autoreload
%autoreload 2
import sys
import os
import types
sys.path.append(os.path.expanduser(os.path.join('~','Documents', 'projects', 'coronavirus')))
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from src import tools
from src.tools import func_log, func_gomp, func_exp
import inspect
import plotly.express as px
from scipy.integrate import odeint
from scipy.optimize import curve_fit
from scipy.optimize import differential_evolution
from src.data_downloader import DATA_REPOS, download_from_repo
from matplotlib.ticker import ScalarFormatter
import matplotlib.ticker as ticker
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### update data from repos

In [2]:
dest=os.path.expanduser(os.path.join('~','Documents', 'projects', 'coronavirus', 'data'))

In [3]:
DATA_REPOS['italy']['streams']

['/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv',
 '/dati-regioni/dpc-covid19-ita-regioni.csv',
 '/dati-province/dpc-covid19-ita-province.csv']

In [4]:
download_from_repo(DATA_REPOS['world']['url'], filenames=DATA_REPOS['world']['streams'], dest=dest)

last commit  2020-04-04 03:20:17


In [5]:
download_from_repo(DATA_REPOS['italy']['url'], filenames=DATA_REPOS['italy']['streams'], dest=dest)

last commit  2020-04-03 18:01:38


### load data

In [230]:
df_naz = pd.read_csv('../data/dpc-covid19-ita-andamento-nazionale.csv').drop('stato',1)
reg = pd.read_csv('../data/dpc-covid19-ita-regioni.csv')
prov = pd.read_csv('../data/dpc-covid19-ita-province.csv').drop('stato',1)
df_naz.index = pd.to_datetime(df_naz.index)
reg['data'] = pd.to_datetime(reg['data'])
prov['data'] = pd.to_datetime(prov['data'])
df_world_confirmed = pd.read_csv('../data/time_series_covid19_confirmed_global.csv')
df_world_deaths = pd.read_csv('../data/time_series_covid19_deaths_global.csv')
df_world_recovered = pd.read_csv('../data/time_series_covid19_recovered_global.csv')
populations = pd.read_csv('../data/API_SP.POP.TOTL_DS2_en_csv_v2.csv', skiprows=4, engine='python').set_index('Country Name')['2018']
df_world_confirmed['pop'] = df_world_confirmed['Country/Region'].map(populations)
df_world_deaths['pop'] = df_world_deaths['Country/Region'].map(populations)
df_world_recovered['pop'] = df_world_recovered['Country/Region'].map(populations)
df_naz = tools.add_extra_features(df_naz)
regions = reg.groupby('denominazione_regione')
df_reg = {}
df_reg['Italy'] = df_naz
for item in regions.groups:
    df_reg[item] = tools.add_extra_features(regions.get_group(item)).replace((np.inf, np.nan), 0)

provinces = prov.groupby('sigla_provincia')
df_prov = pd.DataFrame()
for item in provinces.groups:
    df_prov = pd.concat((df_prov,tools.add_extra_features(provinces.get_group(item)).replace((np.inf, np.nan), 0)),0)

In [231]:
data_columns = ['ricoverati_con_sintomi','terapia_intensiva','totale_ospedalizzati','isolamento_domiciliare','totale_positivi',
           'variazione_totale_positivi','nuovi_positivi','dimessi_guariti','deceduti','totale_casi','tamponi',
           'delta_totale_casi','%delta_totale_casi','growth_factor','delta_dimessi_guariti','%delta_dimessi_guariti',
           'delta_deceduti','%delta_deceduti','deceduti_su_tot','deceduti_su_dimessi']
prov_data_columns = ['totale_casi', 'delta_totale_casi', '%delta_totale_casi', 'growth_factor']
model_columns = ['ricoverati_con_sintomi', 'terapia_intensiva', 'totale_ospedalizzati', 'isolamento_domiciliare', 
                 'totale_positivi','nuovi_positivi', 'dimessi_guariti', 'deceduti', 'delta_deceduti', 'totale_casi', 
                 'tamponi', 'deceduti_su_tot', 'deceduti_su_dimessi']
models = {'gompertz': func_gomp, 'logistic':func_log, 'exponential':func_exp, 'no_fit':'actual'}

## Italy

### provinces analysis

In [602]:
@interact
def get_top_provinces(label= widgets.SelectMultiple(description="data",options=prov_data_columns), 
                      top_prov=widgets.IntSlider(min=1,max=20,step=1,value=10), date=widgets.DatePicker(
                      description='Pick a Date',value=pd.to_datetime(df_prov.index.max())),
                      show_map=True):
    try:
        df_prov.index = pd.to_datetime(df_prov.index)
        if len(label) == 0:
            label = prov_data_columns[:1]
        label = list(label)
        tempdf = df_prov.loc[str(date)][['sigla_provincia','denominazione_provincia', 'lat', 'long']+ label].sort_values(by=label, 
             ascending=False)[:top_prov].set_index('sigla_provincia')

        if label == ['%delta_totale_casi']:
            tempdf[label] = tempdf[label]*100.

        if show_map:
            county_geojson = json.load(open('../data/geodata/italy_prov.json'))
            geo_data = folium.TopoJson(county_geojson, object_path='objects.ITA_adm2')

#             mult=5000.
#             italy_map = folium.Map(location=[41.89,12.48], 
#                                    tiles="cartodbpositron", zoom_start=6, max_zoom = 10, min_zoom = 2,
#                                   title='test')
#             geo_data.add_to(italy_map)
#             # iterate over all the rows of confirmed_df to get the lat/long
#             for row,index in tempdf.iterrows():
#                 #print(np.int(tempdf.loc[row,label]))
#                 folium.Circle(
#                     location=[tempdf.loc[row]['lat'], tempdf.loc[row]['long']],
#                     fill=True,
#                     radius=int(np.log1p(tempdf.loc[row,label].astype('float'))++1.000001)*mult,
#                     color='yellow',
#                     fill_color='indigo',
#                 ).add_to(italy_map)
#             return italy_map

            fig = px.density_mapbox(tempdf, 
                        lat='lat', lon='long', z=label[0], radius=10, 
                        labels = label[0],
                        hover_name='denominazione_provincia',
                        zoom=4.5,  height=800,
                        mapbox_style="carto-positron",
                        title='top {} provinces on day {}'.format(top_prov, date.strftime("%m/%d/%Y"))
                                   )
            fig.show()
        
#             geojson = px.data.election_geojson()
#             fig = go.Figure()
#             for row,index in tempdf.iterrows():
#                 fig.add_traces(go.choropleth(tempdf.reset_index().loc[row], 
#                                 geojson=geo_data.data, 
#                                 locationmode = 'ISO-3',
#                                 lat=tempdf.reset_index().loc[row,'lat'], lon=tempdf.reset_index().loc[row,'long'],
#                                 locations='denominazione_provincia',
#                                 color=label[0],
#                                 featureidkey='objects.ITA_adm2.NAME_2',
#                                 projection="mercator"
#                                ))
#                 fig.update_geos(fitbounds="locations", visible=False)
#                 fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
#             fig.show()
        else:
            tempdf[label].plot(kind='barh', title='top {} provinces on day {}'.format(top_prov, date.strftime("%m/%d/%Y")))
    except Exception as e:
        print(e)

interactive(children=(SelectMultiple(description='data', options=('totale_casi', 'delta_totale_casi', '%delta_…

In [603]:
@interact
def get_prov_data(label=prov_data_columns, region = list(df_prov.denominazione_regione.unique()),date=widgets.DatePicker(
                      description='Pick a Date',value=pd.to_datetime(df_prov.index.max()))):
    try:
        df_prov.index = pd.to_datetime(df_prov.index)
        df_prov.groupby('denominazione_regione').get_group(region).loc[date
            ].set_index('denominazione_provincia')[label].sort_values().plot(kind='barh', 
                title='{} on day {}'.format(label, date.strftime("%m/%d/%Y")))
    except:
        print('data not found')

interactive(children=(Dropdown(description='label', options=('totale_casi', 'delta_totale_casi', '%delta_total…

#### experimental REMOVE

In [392]:
import geopandas as gpd
prov_map = gpd.read_file('../data/geodata/ITA_ADM2.shp')#.to_crs(epsg='4326')
def map_rename(mapdf, old, new):
    for i, oldname in enumerate(old):
        mapdf.loc[mapdf['Name']==oldname, 'Name'] = new[i]
        
map_rename(prov_map,
    ["Forli'-Cesena", 'Carbonia-Iglesias', 'Medio Campidano', 'Ogliastra', 'Olbia-Tempio'], 
    ["Forlì-Cesena", "Sud Sardegna", "Sud Sardegna",  "Sud Sardegna", "Sud Sardegna"])

In [393]:
county_geojson = json.load(open('../data/geodata/italy_prov.json'))
geo_data = folium.TopoJson(county_geojson, object_path='objects.ITA_adm2')

In [560]:
county_geojson['objects']['ITA_adm2']

{'type': 'GeometryCollection',
 'geometries': [{'arcs': [[0, 1, 2, 3, 4]],
   'type': 'Polygon',
   'properties': {'ISO': 'ITA',
    'NAME_0': 'Italy',
    'ID_1': 1,
    'NAME_1': 'Abruzzo',
    'ID_2': 1,
    'NAME_2': 'Chieti',
    'HASC_2': 'IT.CH',
    'TYPE_2': 'Provincia',
    'ENGTYPE_2': 'Province',
    'VARNAME_2': '',
    'style': {}}},
  {'arcs': [[5, -3, 6, 7, 8, 9, 10]],
   'type': 'Polygon',
   'properties': {'ISO': 'ITA',
    'NAME_0': 'Italy',
    'ID_1': 1,
    'NAME_1': 'Abruzzo',
    'ID_2': 2,
    'NAME_2': "L'Aquila",
    'HASC_2': 'IT.AQ',
    'TYPE_2': 'Provincia',
    'ENGTYPE_2': 'Province',
    'VARNAME_2': 'Aquila',
    'style': {}}},
  {'arcs': [[11, -4, -6, 12]],
   'type': 'Polygon',
   'properties': {'ISO': 'ITA',
    'NAME_0': 'Italy',
    'ID_1': 1,
    'NAME_1': 'Abruzzo',
    'ID_2': 3,
    'NAME_2': 'Pescara',
    'HASC_2': 'IT.PE',
    'TYPE_2': 'Provincia',
    'ENGTYPE_2': 'Province',
    'VARNAME_2': '',
    'style': {}}},
  {'arcs': [[13, -13, 

In [551]:
geo_data.data['objects']['ITA_adm2']

{'type': 'GeometryCollection',
 'geometries': [{'arcs': [[0, 1, 2, 3, 4]],
   'type': 'Polygon',
   'properties': {'ISO': 'ITA',
    'NAME_0': 'Italy',
    'ID_1': 1,
    'NAME_1': 'Abruzzo',
    'ID_2': 1,
    'NAME_2': 'Chieti',
    'HASC_2': 'IT.CH',
    'TYPE_2': 'Provincia',
    'ENGTYPE_2': 'Province',
    'VARNAME_2': '',
    'style': {}}},
  {'arcs': [[5, -3, 6, 7, 8, 9, 10]],
   'type': 'Polygon',
   'properties': {'ISO': 'ITA',
    'NAME_0': 'Italy',
    'ID_1': 1,
    'NAME_1': 'Abruzzo',
    'ID_2': 2,
    'NAME_2': "L'Aquila",
    'HASC_2': 'IT.AQ',
    'TYPE_2': 'Provincia',
    'ENGTYPE_2': 'Province',
    'VARNAME_2': 'Aquila',
    'style': {}}},
  {'arcs': [[11, -4, -6, 12]],
   'type': 'Polygon',
   'properties': {'ISO': 'ITA',
    'NAME_0': 'Italy',
    'ID_1': 1,
    'NAME_1': 'Abruzzo',
    'ID_2': 3,
    'NAME_2': 'Pescara',
    'HASC_2': 'IT.PE',
    'TYPE_2': 'Provincia',
    'ENGTYPE_2': 'Province',
    'VARNAME_2': '',
    'style': {}}},
  {'arcs': [[13, -13, 

In [569]:
tempdf.reset_index().iloc[0]

sigla_provincia                 MI
denominazione_provincia     Milano
lat                        45.4668
long                       9.19035
totale_casi                  10391
Name: 0, dtype: object

### regional analysis

In [604]:
@interact
def get_values_for_day(regions = widgets.SelectMultiple(description="regions",options=list(df_reg.keys())),
                       labels = widgets.SelectMultiple(description="data",options=data_columns),
                       date=widgets.DatePicker(description='Pick a Date',value=pd.to_datetime(df_prov.index.max()))):
    if len(regions) == 0:
        regions = ['Italy']
    regions = list(regions)    
    if len(labels) == 0:
        labels = ['nuovi_positivi', 'delta_deceduti','delta_dimessi_guariti']
    labels = list(labels)
    fig = go.Figure()
    for region in regions:
        
        for item in labels: 
            df_reg[region].index = pd.to_datetime(df_reg[region].index)
        fig.add_traces(go.Bar(y=labels, x=df_reg[region][labels].loc[date], name=region, orientation='h'))
        fig.update_layout(showlegend=True,title='day ' + str(date.strftime("%m/%d/%Y")))

    fig.show()

interactive(children=(SelectMultiple(description='regions', options=('Italy', 'Abruzzo', 'Basilicata', 'Calabr…

In [605]:
@interact
def plt_region(regions = widgets.SelectMultiple(description="regions",options=list(df_reg.keys())), 
               labels = widgets.SelectMultiple(description="data",options=data_columns),
              log=False,
              relative_dates=False):    
    
    if len(labels) == 0:
        labels = data_columns[:1] 
    labels = list(labels)
    if len(regions) == 0:
        regions = ['Italy']
    regions = list(regions)    
    fig = go.Figure()
 
    for item in labels:
        for region in regions:
            df_reg[region].index = pd.to_datetime(df_reg[region].index)
            temp = df_reg[region]
            if relative_dates: temp = temp.loc[~(temp[label]==0)].reset_index(drop=True).iloc[:-1] 
            if log:
                fig.add_traces(go.Scatter(x=temp.index, y=temp[item], name=item+'_'+region))
            else:
                fig.add_traces(go.Bar(x=temp.index, y=temp[item], name=item+'_'+region))
    fig.update_layout(showlegend=True)
    if log: fig.update_layout(yaxis_type="log")
    fig.show()

interactive(children=(SelectMultiple(description='regions', options=('Italy', 'Abruzzo', 'Basilicata', 'Calabr…

### Logistic model evolution

In [606]:
@interact
def get_model(region=['Italy']+list(df_reg.keys()), 
              start_fit=widgets.DatePicker(value=pd.to_datetime(df_naz.index[0])), 
              end_fit=widgets.DatePicker(value=pd.to_datetime(df_naz.index[-1])), fwd_look=50, 
              func=models, label = model_columns, stdev=widgets.IntSlider(min=0, max=3, value=1)):
    if region=='Italy':
        df = df_naz
    else:
        df = df_reg[region]
        df.index = pd.to_datetime(df.index)
    start_fit = pd.Timestamp(start_fit)
    end_fit = pd.Timestamp(end_fit)
    y_fit = df[label].loc[start_fit:end_fit]
    x_fit = range(len(y_fit.index))
    if isinstance(func, types.FunctionType):
        x_pred2 = range(len(df.index)+fwd_look)
        x_pred1 = range(len(df.index))
        sig = inspect.signature(func)
        n_params = len(sig.parameters.items()) -1
    
        params, params_cov = curve_fit(func, x_fit, y_fit, 
                            bounds=([0. for item in range(n_params)], [np.inf for item in range(n_params)]), method='trf')
        stderr = np.sqrt(np.diag(params_cov))
        params_up = params + stderr * stdev
        params_down = params - stderr * stdev
        y_pred1 = func(x_pred1, *params)
        y_pred2 = func(x_pred2, *params)
        y_pred_up = func(x_pred2, *params_up)
        y_pred_down = func(x_pred2, *params_down)
        errors = (y_pred_up - y_pred_down)
        rmse = np.sqrt(np.mean((y_fit - func(x_fit, *params)) ** 2))
    fig = go.Figure()
    if isinstance(func, types.FunctionType):
        fig.add_traces(go.Scatter(x=pd.date_range(start=df.index.min(), end=df.index.max()), y=df[label].values, 
                              name=label, mode='markers'))
        fig.add_traces(go.Scatter(x=pd.date_range(start=start_fit, end=end_fit), 
                              y=y_pred1[:(end_fit-start_fit).days], name='model rmse: '+str(int(rmse))))
        fig.add_traces(go.Scatter(x=pd.date_range(start=end_fit, end=df.index.max()+pd.Timedelta(str(fwd_look)+'d')), 
                              y=y_pred2[(end_fit-start_fit).days:],error_y=dict(array=errors,color='green',
                                    thickness=.2,width=0.5), name='forecast'))
    else:
        fig.add_traces(go.Bar(x=pd.date_range(start=df.index.min(), end=df.index.max()), y=df[label].values, 
                              name=label))


    fig.update_layout(showlegend=True)
    fig.show()

interactive(children=(Dropdown(description='region', options=('Italy', 'Italy', 'Abruzzo', 'Basilicata', 'Cala…